In [1]:
import toml
import os
import pandas as pd
import cv2

In [2]:
dict_toml = toml.load(open('../config.toml'))

In [3]:
image_path = dict_toml["path"]["dataset"]["image"]

In [4]:
image_df = pd.read_table("../../list_attr_celeba.txt", header=0,sep=" ", index_col=0)

In [5]:
image_df["Male"]["000001.jpg"]

-1

In [6]:
f = open('../not_detected.txt', 'r')

datalist = f.read().splitlines()
f.close()

In [7]:
data = [not i in datalist for i in image_df.index]

In [8]:
image_df[data]["Male"]

000001.jpg   -1
000002.jpg   -1
000003.jpg    1
000005.jpg   -1
000007.jpg    1
             ..
202595.jpg   -1
202596.jpg    1
202597.jpg    1
202598.jpg   -1
202599.jpg   -1
Name: Male, Length: 182367, dtype: int64

In [9]:
#os.listdir("../../voice/train/F/")

In [10]:
for i in os.listdir("../../voice/train/F/VCC2SF1"):
    a = pd.read_pickle("../../voice/train/F/VCC2SF1/"+i)
    print(a.shape)

FileNotFoundError: [Errno 2] No such file or directory: '../../voice/train/F/VCC2SF1'

# dataloader

## transform

### voice

In [11]:
class VoiceTrans(object):
    def __init__(self, maxi, mini):
        self.maxi = maxi
        self.mini = mini

    def norm_voice(self, array):
        array -= self.mini
        array /= self.maxi
    
        return array
    
    def cut(self, voice):
        return voice[:, :voice.shape[1]-voice.shape[1]%4]
    
    def __call__(self, sample):
        
        return self.cut(self.norm_voice(sample))

In [12]:
voice = pd.read_pickle("../../voice/train/VCC2SF1/10001.pkl")

In [ ]:
#t = VoiceTrans()
#t(voice).shape

### image

In [13]:
class ImageTrans(object):
    def __init__(self):
        pass

    def norm_image(self, array):
        return array/255
    
    def __call__(self, sample):
        
        return self.norm_image(sample).T

In [14]:
image = cv2.imread("../../image/000001.jpg")

In [15]:
a = ImageTrans()
a(image).shape

(3, 32, 32)

# dataset

In [16]:
import pathlib
import itertools
import pandas as pd
import numpy as np
import torch

### voice

In [17]:
class VoiceDataset(torch.utils.data.Dataset):
    def __init__(self, path, train, trans):
        p = pathlib.Path(path)
        
        if train:
            self.path = p / "train"
        else:
            self.path = p / "eval"
        
        self.trans = trans
        
        
        self.dir = [i for i in self.path.iterdir() if i.is_dir()]       
        self.file = list(itertools.chain.from_iterable([[j for j in i.iterdir()] for i in self.dir]))
        
        self.data = [pd.read_pickle(i) for i in self.file]
        self.label = [1 if "M" == str(i.parent)[-2] else -1 for i in self.file]
    
        self.datanum = len(self.data)
    
    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.label[idx]

        if self.trans:
            k = np.concatenate(c.data,axis=1)
            maxi = k.max()
            mini = k.min()
            self.transform = VoiceTrans(maxi, mini)
            
            out_data = torch.tensor(self.transform(out_data))

        return out_data, out_label

In [18]:
c = VoiceDataset(path="../../voice", train=1, trans=1)

In [19]:
c.__getitem__(1).shape

AttributeError: 'tuple' object has no attribute 'shape'

### face

In [20]:
import random

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, path, trans):
        p = pathlib.Path(path)
        if trans:
            self.transform = ImageTrans()
        
        self.dir = p
        self.file = [i for i in self.dir.iterdir() if i.suffix == ".jpg"]
        
        
        if self.transform:
            self.data = [self.transform(cv2.imread(str(i))) for i in self.file]
        else:
            self.data = [cv2.imread(str(i)) for i in self.file]
 
        
        image_df = pd.read_table("../../list_attr_celeba.txt", header=0,sep=" ", index_col=0)
        self.label = [image_df["Male"][i.name] for i  in self.file]
        """
        for i in self.file:
            try:
                image_df["Male"][i.name]
            except:
                print(i.name)
        """     
                
        
        self.datanum = len(self.data)
    
    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.label[idx]

        
        return out_data, out_label
    
    def sample_label(self, male, num):
        l = random.sample([i for i, x in enumerate(self.label) if x==male], num)
        return l
    
    def sample_data(self, male, num):
        label = self.sample_label(male, num)
        data = self.data
        print(data[0].shape)
        return torch.tensor([data[i] for i in label])
    
    
    
    

In [21]:
f = ImageDataset("../../image", trans=1)

## dataloader

In [ ]:
def collate_fn(batch):
    # bactchはDatasetの返り値 (タプル) のリスト
    images, labels = [], []
    
    for image, label in batch:
        images.append(image)
        labels.append(label)
        
    images = torch.stack(images, dim=0)

    # labelsはTensorリストのまま

    return images, labels

In [22]:
trainloader = torch.utils.data.DataLoader(f, batch_size = 2, shuffle = False, num_workers = 2)

In [23]:
for data,label in trainloader:
    break
print(data.shape)
print(label.shape)

torch.Size([2, 3, 32, 32])
torch.Size([2])


In [24]:
f.sample_data(1, 10).shape

(3, 32, 32)


torch.Size([10, 3, 32, 32])